In this ipynb we would like to judge how does prior beliefs affect the algorithm, for that, we pass a Laplacian with zeros where we want the algorithm to not learn any edge, we will do a plot

for N=10, N=20
for ER, BA, RBF
percentage of prior knowledge vs f-measure with a 1/(N-1) threshold of weights (or with no threshold??)

In [1]:
import sys
sys.path.append('../')
import learnHeat as lh
import learnHeat3 as lh3
from importlib import reload
lh3 = reload(lh3)
lh = reload(lh)
import numpy as np
import matplotlib.pyplot as plt
import scores_table as scort

In [18]:
trial = np.load("trials.npy")
trial += 1
steps = 4
prior = np.linspace(0,1,steps)
sims = 1
N = 7
tau0 = [2.5,4]
alpha = 0.01
beta = 0.1
max_iter = 50
# we have 3 graphs, 10 sims and 11 priors
scores = np.zeros([3,sims,steps])

for i in range(sims):
    print (i, end="\r")
    # create graphs
    BA = lh.heat_graph_BA(N)
    ER = lh.heat_graph_ER(N)
    RBF = lh.heat_graph_RBF(N)
    XBA = lh.create_deltas(BA,tau0)
    XER = lh.create_deltas(ER,tau0)
    XRBF = lh.create_deltas(RBF,tau0)
    # do the learning with different prior knowledge
    for j in range(steps):
        print(f"Prior {prior[j]}", end="\r")
        # learn the graphs with prior
        # this is not working, it should 
        res1 = lh3.learn_heat(XBA,lh3.cover_zeros(BA,prior[j]),tau0=tau0,alpha=alpha,beta=beta,max_iter=max_iter)
        res2 = lh3.learn_heat(XER,lh3.cover_zeros(ER,prior[j]),tau0=tau0,alpha=alpha,beta=beta,max_iter=max_iter)
        res3 = lh3.learn_heat(XRBF,lh3.cover_zeros(RBF,prior[j]),tau0=tau0,alpha=alpha,beta=beta,max_iter=max_iter)
        # threshold a bitttt
        LeBA = res1["L"]
        LeBA[abs(LeBA)<0.001] = 0
        LeER = res2["L"]
        LeER[abs(LeER)<0.001] = 0
        LeRBF = res3["L"]
        LeRBF[abs(LeRBF)<0.001] = 0
        # compute the fmeasure and appendscoresz
        scores[0,i,j] = scort.scores(LeBA,BA)
        print(scores)
        scores[1,i,j] = scort.scores(LeER,ER)
        print(scores)
        scores[2,i,j] = scort.scores(LeRBF,RBF)
        print(scores)

np.save("justincase.npy",scores)
np.save(f"{trial}_sims_{sims}_steps_{steps}_maxiter_{max_iter}_alpha_{alpha}_beta_{beta}.npy",scores)
np.save("trials.npy",trial)

/home/aner/Documents/math/learning_graphs/prior_knowledge/../learnHeat3.py:145: RuntimeWarning: invalid value encountered in scalar divide
  B[i,j] = (np.exp(Eval[i])-np.exp(Eval[j])) / (Eval[i]-Eval[j])
/home/aner/.local/lib/python3.8/site-packages/cvxpy/problems/problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/home/aner/Documents/math/learning_graphs/prior_knowledge/../learnHeat3.py:36: RuntimeWarning: invalid value encountered in scalar multiply
  c2/2*(np.linalg.norm(Ltp1-Lt, 'fro')**2)


Failure:interrupted


SolverError: Solver 'SCS' failed. Try another solver, or solve with verbose=True for more information.

In [3]:
np.save(f"{trial}_sims_{sims}_steps_{steps}_maxiter_{max_iter}_alpha_{alpha}_beta_{beta}.npy",scores)

In [8]:
Lstar = lh3.cover_zeros(BA,0)
np.count_nonzero(Lstar)

58

In [10]:
res4 = lh3.learn_heat(XBA,Lstar,tau0=tau0,alpha=alpha,beta=beta,max_iter=max_iter)

In [17]:
np.count_nonzero(res4["L"])
Lstar2 = res4["L"]
Lstar2[abs(Lstar2)<0.001] = 0
np.count_nonzero(Lstar2)

58